


## 1. KR-BERTopic
[참조]
BERTopic :https://github.com/MaartenGr/BERTopic/tree/05a6790b21009d1704e912e0d9ae22290694cfed
BERTopic 개념 : https://wikidocs.net/162076

### 1-1. BERTopic
BERTopic은 BERT embeddings과 클래스 기반(class-based) TF-IDF를 활용하여 주제 설명에서 중요한 단어를 유지하면서도 쉽게 해석할 수 있는 조밀한 클러스터를 만드는 토픽 모델링 기술이다. BERTopic 알고리즘은 크게 세 가지 과정을 거친다.
1) 텍스트 데이터를 SBERT로 임베딩한다.
SBERT를 사용하여 문서를 임베딩한다. 이때, BERTopic은 기본적으로 아래의 BERT들을 사용한다.
•	"paraphrase-MiniLM-L6-v2" : 영어 데이터로 학습된 SBERT
•	"paraphrase-multilingual-MiniLM-L12-v2" : 50개 이상의 언어로 학습된 다국어 SBERT
2) 문서를 군집화한다.
UMAP을 사용하여 임베딩의 차원을 줄이고 HDBSCAN 기술을 사용하여 차원 축소된 임베딩을 클러스터링하고 의미적으로 유사한 문서 클러스터를 생성한다.
3) 토픽 표현을 생성한다
마지막 단계는 클래스 기반 TF-IDF로 토픽을 추출한다.





## 2. 토픽 모델링
[참조]
KoBERTopic 코드 : https://github.com/ukairia777/KoBERTopic
자주 사용되는 불용어 링크 : https://www.ranks.nl/stopwords/korean




형태소 분석기 spacy </br>
단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩 벡터를 만드는 CounterVectorizer, TfidfVectorizer </br>
토픽 모델링을 위한 BERTopic 라이브러리를 import 한다.  

In [1]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy 
from bertopic import BERTopic
import requests
import pandas as pd
from io import StringIO

In [4]:
df=pd.read_excel('빅카인즈_뉴스.xlsx')
df=df.loc[:,['일자','제목','본문']]

In [15]:
print(df.shape)
df.head()

(17331, 3)


,일자,제목,본문
0,20230214,"[클릭 e종목]""현대백화점, 실적 부진 목표가↓""",[아시아경제 손선희 기자] 지난해 부진한 실적을 기록한 현대백화점에 대해 증권사들이...
1,20230214,"필옵틱스, 지난해 영업익 177억 흑자전환",[헤럴드경제=증권부] 필옵틱스는 지난해 연결기준 영업이익이 176억9866만원으로 ...
2,20230214,"펄어비스, 지난해 영업익 166억원 전년 대비 61.4%↓","펄어비스가 지난해 매출 3860억원, 영업이익 166억원, 당기순손실 411억원을 ..."
3,20230214,주총 앞둔 삼성전자 이재용 등기임원 복귀 관심,[아시아경제 박선미 기자]한달 뒤 열리는 삼성전자 정기 주주총회에서 이재용 삼성전자...
4,20230214,"현대차 기아, 1월 인도 역대 최다 판매 “전기차 시장도 공략한다”",[헤럴드경제=정찬수 기자] 현대자동차와 기아가 지난달 인도에서 월간 최다 판매기록을...


In [16]:
#중복 제거
kr_news = df.drop_duplicates(subset=['본문'],keep='first').reset_index(drop=True)
print(kr_news.shape)

(16846, 3)


In [17]:
#BERTopic의 경우 텍스트 전처리를 사전에 진행하는 것보다 후에 처리하는 것이 더 좋은 결과를 뽑아낼 수 있습니다.
import re
kr_news.제목=kr_news.제목.apply(lambda x : re.sub('[^A-Za-z0-9가-힣]', ' ', x))
kr_news.제목=kr_news.제목.apply(lambda x : re.sub(' +', ' ', x))
kr_news.본문=kr_news.본문.apply(lambda x : re.sub('[^A-Za-z0-9가-힣]', ' ', x))
kr_news.본문=kr_news.본문.apply(lambda x : re.sub(' +', ' ', x))



In [30]:
kr_news.일자=kr_news.일자.astype(str)
kr_news.일자=pd.to_datetime(kr_news.일자)

In [31]:
print(kr_news.shape)
kr_news.head()

(16846, 3)


,일자,제목,본문
0,2023-02-14,클릭 e종목 현대백화점 실적 부진 목표가,아시아경제 손선희 기자 지난해 부진한 실적을 기록한 현대백화점에 대해 증권사들이 ...
1,2023-02-14,필옵틱스 지난해 영업익 177억 흑자전환,헤럴드경제 증권부 필옵틱스는 지난해 연결기준 영업이익이 176억9866만원으로 전...
2,2023-02-14,펄어비스 지난해 영업익 166억원 전년 대비 61 4,펄어비스가 지난해 매출 3860억원 영업이익 166억원 당기순손실 411억원을 기록...
3,2023-02-14,주총 앞둔 삼성전자 이재용 등기임원 복귀 관심,아시아경제 박선미 기자 한달 뒤 열리는 삼성전자 정기 주주총회에서 이재용 삼성전자...
4,2023-02-14,현대차 기아 1월 인도 역대 최다 판매 전기차 시장도 공략한다,헤럴드경제 정찬수 기자 현대자동차와 기아가 지난달 인도에서 월간 최다 판매기록을 ...


In [57]:
kr_news=kr_news[kr_news.일자.between('2023-02-12','2023-02-14')].sort_values('일자') #원하는 날짜 조정 (주간 단위)
kr_news=kr_news.reset_index(drop=True)

In [59]:
documents = [line.strip() for line in kr_news['본문']]

In [60]:
preprocessed_documents = []
i = 0 
deleted_article = []


for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)
    i += 1 
  
  else: 
    deleted_article.append(i)
    i +=1

100%|██████████████████████████████████████████████████████████████████████████| 4694/4694 [00:00<00:00, 391272.76it/s]


### 2-1. 단어 빈도수를 기반으로한 불용어 찾기 

: 단어 빈도수가 높은데 중요하지 않은 단어와 일반적으로 자주 사용되는 단어를 합쳐서 불용어 만들었습니다.

 한나눔 토크나이저 사용해서 토큰화 한 뒤 단어 빈도수가  상위 200 안에 드는 단어들을 기준으로 불용어 선택

- 자주 사용되는 불용어 링크 : 
https://www.ranks.nl/stopwords/korean

- 추가로 몇가지 불용어 리스트를 직접 다운받아오고 너무 많이 나오는 단어를 제거해줬습니다

In [61]:
with open('./data/stopwords (1).txt','r',encoding='utf-8-sig') as f:
    stopwords_list=[]
    example =f.readlines()
    for line in example:
        stopwords_list.append(line.strip())

In [62]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 

korean_stopwords=' 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓'
stop_words = korean_stopwords

stop_words = stop_words.split(' ')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82103\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
stop_words=stop_words+stopwords_list

In [64]:
stop_words.extend(['공시됐다고','행사됐다고','제기됐다고'])

기사 전체를 전처리하여 데이터를 정제했다

### 2-2.	Spacy customtokenizer

#### CustomTokenizer 생성 

In [65]:
# 코랩, 첫 코드 실행시 설치 
#!python -m spacy download ko_core_news_lg 
nlp = spacy.load("ko_core_news_lg")

In [66]:
#명사와 동사만 뽑아낼 수 있도록 처리 여기서는 명사만 뽑히게함.
def tokenizer(text):
        doc=nlp(text)
    
        lemmas = []
        texts_list = [token.lemma_.split('+') for token in doc]
        tags_list = [token.tag_.split('+') for token in doc]
        

        for token_idx in range(len(doc)):
            if tags_list[token_idx][0] in ('ncn','ncpa','ncps','nc','nq','f'): #'pvg','paa',
                if len(texts_list[token_idx])==len(tags_list[token_idx]):
                    if tags_list[token_idx][0].startswith('n') or tags_list[token_idx][0].startswith('f'):
                        lemmas.append(''.join([text for tag,text in zip(tags_list[token_idx],texts_list[token_idx]) if tag in ('ncn','ncpa','ncps','nc','nq','f')]).replace(' ','_'))
                    else:
                        lemmas.append(texts_list[token_idx][0]+'다')
                elif tags_list[token_idx][0] in ('ncn','ncpa','ncps','nc','f'):
                    lemmas.append(texts_list[token_idx][0])
                else:
                    lemmas.append(''.join((texts_list[token_idx])).replace(' ','_'))
                

        return lemmas

#### TfidfVectorizer로 BOW(Bag of Words) 생성 

In [67]:
vectorizer = TfidfVectorizer(tokenizer=tokenizer,min_df=2, ngram_range=(1,1),stop_words=stop_words) # max_features 정수, 기본값=없음
# 없음이 아닌 경우 코퍼스 전체에서 용어 빈도로 정렬된 상위 max_features만 고려하는 어휘를 작성하는 것 

In [68]:
vectorizer.fit(kr_news.본문)
vectorizer.vocabulary_

{'전경전남': 7383,
 '올해': 6244,
 '전라남도': 7422,
 '지원사업': 8182,
 '참여기업': 8349,
 '모집': 3309,
 '중소기업': 8027,
 '취업': 8735,
 '청년': 8460,
 '장기': 7231,
 '근속': 1807,
 '유도': 6548,
 '기업': 1978,
 '장려금': 7238,
 '지급': 8112,
 '사업': 4314,
 '대상': 2580,
 '공고일': 1360,
 '기준': 2021,
 '전남': 7407,
 '주소': 7904,
 '이하': 6861,
 '근로자': 1802,
 '방탄소년단': 3705,
 'bts': 132,
 '소속사': 4951,
 '하이브': 9590,
 'sm엔터테인먼트': 603,
 '인수전': 6914,
 '공정거': 1441,
 '이수만': 6783,
 'sm': 595,
 '대주주': 2627,
 '총괄': 8553,
 '대규모': 2540,
 '주식': 7907,
 '인수': 6913,
 '소액': 4961,
 '지분': 8139,
 '변수': 3849,
 '공정': 1440,
 '공정위': 1445,
 '심사': 5584,
 '통과': 9065,
 '가능': 693,
 'm': 455,
 'a': 74,
 '확실시': 10118,
 '파이낸셜뉴스': 9210,
 '금융감독원': 1842,
 '금융소비자': 1857,
 '리포터': 3071,
 '도입': 2749,
 '불합리': 4141,
 '금융': 1841,
 '관행': 1517,
 '개선': 900,
 '제도': 7704,
 '과정': 1473,
 '소비자': 4936,
 '의견': 6679,
 '청취': 8489,
 '일반공모': 6995,
 '기관추천': 1917,
 '지식': 8154,
 '금융거래': 1843,
 '카드': 8786,
 '업계': 5880,
 '연체율': 6064,
 '상승세': 4511,
 '금융권': 1847,
 '삼성': 4444,
 'kb국민': 383,
 '우리카

In [69]:
from umap import UMAP
import hdbscan

umap_model = UMAP(n_neighbors=15, min_dist=0.1, n_components=5,random_state=42,metric='cosine')
hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [70]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 top_n_words=15,
                 min_topic_size=10,
                 umap_model=umap_model,hdbscan_model=hdbscan_model,
                 verbose=True)

## model save 하기

In [71]:
topics, probs = model.fit_transform(kr_news.본문)

Batches:   0%|          | 0/147 [00:00<?, ?it/s]

2023-02-14 10:22:06,369 - BERTopic - Transformed documents to Embeddings
2023-02-14 10:22:14,787 - BERTopic - Reduced dimensionality
2023-02-14 10:22:15,045 - BERTopic - Clustered reduced embeddings


In [72]:
# get_topic_info() : 토픽의 개수, 토픽의 크기, 각 토픽에 할당된 단어들을 일부 볼 수 있음
# -1은 토픽이 할당되지 않은 모든 이상치 문서(outliers)들을 나타냄 

model.get_topic_info()[:50]

,Topic,Count,Name
0,-1,2066,-1_기자_지원_갤럭시_지난해
1,0,588,0_학위_승진_ls그룹_서울
2,1,195,1_대통령_윤_은행_국민
3,2,106,2_전기차_기아_판매량_자동차
4,3,104,3_ai_인공지능_반도체_sk텔레콤
5,4,77,4_오아시스_철회_상장_중단
6,5,61,5_매출액_연결기준_증권부_당기
7,6,60,6_cdp_기후변화_탄소경영_시상식
8,7,60,7_아파트_집값_부동산_거래량
9,8,57,8_경유_휘발유_전차_협력업체


In [73]:
model.get_topic(13)# 토픽 하나씩 뽑아서 키워드 체크

[('sm', 0.11378102385987161),
 ('하이브', 0.10183982058497992),
 ('인수', 0.06208952620352555),
 ('sm엔터테인먼트', 0.061878868581501004),
 ('이수만', 0.05647946792884668),
 ('이수', 0.05494152261925628),
 ('k팝', 0.04979379459868963),
 ('bts', 0.04525863430285118),
 ('지분', 0.044239566117139396),
 ('총괄프로듀서', 0.04416708956785344),
 ('프로듀서', 0.042890515816834694),
 ('카카오', 0.03955505467180028),
 ('방탄소년단', 0.03817388907395107),
 ('경영권', 0.037955814523959235),
 ('총괄', 0.037107576326786365)]

In [28]:
ㅓdf=model.get_topic_info()
df.to_csv('10월1주차_한국뉴스_차원축소X.csv')

In [74]:
kr_news['Topics'] = topics

In [75]:
topic_df=model.get_topic_info()

topic_df=pd.merge(kr_news,topic_df,how='left',left_on='Topics',right_on='Topic')



In [77]:
file_path =  "./data/"

file_name = "한국뉴스" + '_토픽별' + ".csv"

file_full_path = file_path + file_name

topic_df.to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함!  

<hr><font color="green"><h1>from file: 2.삼성전자 토픽별 핵심 기사 추출</h1></font>

# 여기서부터는 DACON 삼성전자 뉴스 추천 앱 코드 공유 적용입니다.
#### 원문 글
https://dacon.io/competitions/official/235914/codeshare/5728 

# 1. KR-WordRank
[참조]  

Keyword extraction process : https://github.com/lovit/KR-WordRank

KRWordRank: method for word / keyword extraction

KRWordRank는 Kim et al.(2014)^[1]의 논문을 바탕으로 한 비지도학습기반 단어 추출 기법으로, 데이터기반으로 주요단어 (키워드)를 추출하는 알고리즘이다. 하나의 도메인에 대한 문서들을 바탕으로 명사/형용사/동사/부사 (L set) 중에서 빈도수가 높거나, 주요 단어들과 함께 등장하는 단어를 키워드로 추출한다. KRWordRank는 이름에서 나타나는바와 같이 단어 후보 (subtokens)을 이용하여 word-graph를 생성한 뒤, PageRank의 랭킹 학습 방식을 이용하여 word-graph의 hub subtokens을 추출한다.
KRWordRank는 다음의 가정을 기반으로 단어를 추출한다. 단어 주변에는 단어가 등장하며, 올바른 단어는 주위의 많은 단어들과 연결되어 있다. 그렇기 때문에 단어는 주위 단어들에 의하여 단어 점수가 보강(reinforced)된다.

![img](./image/kr-wordrank1.png)


한국어는 의미를 지니는 단어 집합과 문법 기능을 하는 복합형태소 집합으로 나뉘어지며, [문법/명사] + [을/조사]와 같이 어절의 왼쪽에 의미를 지니는 단어인 명사/형용사/동사가 위치한다. 부사는 그 자체로 한 어절을 이룬다. 그렇기 때문에 KRWordRank는 의미있는 단어로서 어절 자체나 어절의 왼쪽에 등장하는 L set을 추출한다. 또한 한국어는 한 글자에 지나치게 많은 의미가 담겨져 있어 해석이 모호하기 때문에 1음절 단어는 추출되는 단어에서 제외한다. 실제로 subtokens으로 이뤄진 word-graph에서 1음절 단어들은 매우 높은 랭킹을 지닌다. KRWordRank는 아래 그림과 같이 subtokens을 어절의 위치에 따라 L/R tags를 부여하여 word-graph를 만든 뒤, 랭킹을 계산한다.


![img](./image/kr-wordrank2.png)

[논문] 

[1] Kim, H. J., Cho, S., & Kang, P. (2014). KR-WordRank: An Unsupervised Korean Word Extraction Method Based on WordRank. Journal of Korean Institute of Industrial Engineers, 40(1), 18-33.




## 1-1. 핵심 기사 추출 
KR-WordRank 에서는 keywords 를 많이 포함한 문장을 핵심 문장으로 선택한다. 문장을 추출하는 원리는 추출된 키워드의 랭크값을 이용하여 키워드 벡터를 만든 뒤, 코싸인 유사도 기준으로 입력된 문장 벡터가 키워드 벡터와 유사한 문장을 선택하는 것이다.
summarize_with_sentences 함수에 기사 내용 전체 를 입력하면 KR-WordRank 를 학습하여 키워드와 이를 이용한 핵심 문장을 선택한다. 

몇 가지 parameter를 추가할 수 있다. stopwords 는 키워드에서 제거한다. 이들은 키워드 벡터를 만들 때에도 이용되지 않는다. 
또한 키워드 벡터와 유사한 문장을 우선적으로 선택하다 보면 이전에 선택된 문장과 중복되는 문장들이 선택되기도 하는데 이는 diversity 를 이용하여 조절할 수 있다. diversity 는 코사인 유사도 기준 핵심 문장 간의 최소 거리이며 이 값이 클수록 다양한 문장이 선택된다. 

In [78]:
import pandas as pd

In [85]:
df = pd.read_csv('./data/한국뉴스_토픽별.csv')
df = df.drop_duplicates(subset = ["본문"]) # 겹치는 뉴스 제거 
df = df.reset_index(drop=True) # 인덱스 수정 

In [86]:
df.head(2)

,일자,제목,본문,Topics,Topic,Count,Name
0,2023-02-12,장성군 청년근속장려금 참여기업 모집,장성군청 전경전남 장성군이 올해 전라남도 청년근속장려금 지원사업 참여기업을 모집한다...,-1,-1,2066,-1_기자_지원_갤럭시_지난해
1,2023-02-12,SM 품는 하이브 공정위가 키맨 보다 동남아 심사 더 관심,방탄소년단 BTS 의 소속사인 하이브가 SM엔터테인먼트 인수전에 나서면서 공정거래위...,13,13,39,13_sm_하이브_인수_sm엔터테인먼트


In [88]:
!pip install krwordrank
import sys
sys.path.append('../')

from krwordrank.word import KRWordRank
import krwordrank

topic -1은 분류가 되지 않은 기사이기 때문에 topic -1 을 제외한 기사 선택합니다.

In [89]:
df = df[df['Topics']!=-1]
df.reset_index(inplace=True,drop=True)

In [90]:
df.head(4)

,일자,제목,본문,Topics,Topic,Count,Name
0,2023-02-12,SM 품는 하이브 공정위가 키맨 보다 동남아 심사 더 관심,방탄소년단 BTS 의 소속사인 하이브가 SM엔터테인먼트 인수전에 나서면서 공정거래위...,13,13,39,13_sm_하이브_인수_sm엔터테인먼트
1,2023-02-12,러시아 공장 멈추자 신차급 중고차 수출 폭증,지난해 우크라이나와의 전쟁 이후 차량 현지 생산이 멈추거나 수출이 중단된 러시아가 ...,2,2,106,2_전기차_기아_판매량_자동차
2,2023-02-12,초거대 AI 두뇌 2배로 확대 SKT 한국형 챗GPT 속도전,인공지능 AI 연구 스타트업 오픈AI의 챗GPT 공개 이후 국내외 빅테크의 차세대 ...,3,3,104,3_ai_인공지능_반도체_sk텔레콤
3,2023-02-12,인천 2월 아파트분양경기전망 호전 1 3 부동산 대책 영향 분석,인천의 2월 아파트분양경기전망이 크게 호전된 것으로 나타났다 이는 정부의 1 3 부...,40,40,20,40_입주율_아파트_주택산업연구원_하락


#### 토픽별로 기사 모으기

토픽별로 인덱스와 기사를 열로 갖는 빈 데이터프레임을 생성한 뒤 리스트로 묶어줍니다.

In [91]:
df.Topics.nunique()

66

In [92]:
topic_list=[]
for i in range(df.Topics.nunique()):        
        empty_df = pd.DataFrame(columns=['indexes','Articles'])
        topic_list.append(empty_df)
        

각 토픽별 빈 데이터프레임에 기사의 인덱스와 내용을 하나의 행으로 넣습니다. 

In [95]:
for i in range(len(topic_list)):
    index_list = df[df['Topics']==i].index.tolist()
    for j in range(len(index_list)):
        a = df[df['Topics']==i]['제목'].loc[index_list[j]] + ' ' + df[df['Topics']==i]['본문'].loc[index_list[j]]
        topic_list[i].loc[j]=[index_list[j],a]


0번째 토픽의 기사입니다.

In [100]:
topic_list[0].head()

,indexes,Articles
0,9,르포 일요일에도 영업합니다 대구 대형마트 의무휴업일 전환에 북새통 헤럴드경제 ...
1,10,오뚜기 육류소스 9종 패키지 새 구성 친환경 용기 라벨 눈길 머니투데이 김성진 기...
2,17,합산 영업익 4조 에도 웃지 못하는 이통3사 경영과제 산적 이통3사 작년 합산 영...
3,19,5만3천가구 미니신도시로 변신할 목동 서울의 교육특구로 이기재 양천구청장 인터뷰...
4,24,미니신도시 될 목동 서울의 교육특구로 재건축이 급물살을 타고 있는 서울 양천구 ...


토픽별로 기사들을 묶기 위해서 빈 리스트를 만듭니다. 

In [101]:
texts=[]
for i in range(df.Topics.nunique()):        
        empty_list = []
        texts.append(empty_list)
        

각 토픽별로 기사들을 묶어줍니다. 
이때 기사 맨 앞에 인덱스와 '#'을 붙여줍니다. 

In [102]:
for i in range(len(topic_list)):
    for j in range(len(topic_list[i])):
        texts[i].append(str(topic_list[i]['indexes'].loc[j]) +'#'+ topic_list[i]['Articles'].loc[j])

## 토픽별 핵심 기사 추출

토픽별로 묶인 기사들 중 핵심 기사들을 선택하기 위한 함수를 만듭니다 

In [103]:
from krwordrank.sentence import summarize_with_sentences

stopwords =' 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓'

def summarize_sentences(texts):

    keywords, sents = summarize_with_sentences(texts, 
                                               stopwords=stopwords, 
                                               num_keywords=100, 
                                               num_keysents=6,  
                                               diversity=0.5)
    
    return sents

In [104]:
results = list(map(summarize_sentences, texts))

Topic0 의 핵심 기사입니다

In [105]:
results[0]

['55#구자열 LS그룹 의장 명예 철학박사 구자열 LS그룹 이사회 의장 한국무역협회 회장 사진 이 모교인 고려대에서 경영인이 받을 수 있는 최고 영예인 명예철학박사 학위를 받았다 12일 LS에 따르면 고려대는 지난 10일 서울 성북구 고려대 100주년기념삼성관에서 정진택 총장 이관영 대학원장을 비롯한 관계자들이 참석한 가운데 구 의장에게 명예철학박사 학위를 수여했다 서진우 기자 ',
 '1733# BNK금융 계열사 CEO 후보군 선정 안감찬 이두호 용퇴   사진 BNK금융그룹 BNK금융그룹은 주요 자회사인 부산은행 경남은행 BNK캐피탈 최고경영자 CEO 선임을 위한 임원후보추천위원회 임추위 를 개최해 1차 후보군을 선정했다고 13일 밝혔다 BNK에 따르면 이들 자회사는 앞서 지난 10일까지 최고경영자 후보군을 대상으로 지원서 제출을 요청한 바 있다 임추위는 지원서를 제출한 후보 ',
 '2057#김포시 지역 경제 견인할 기업지원 로드맵 공개 김병수 김포시장이 지난달 13일 진행된 김포상공회의소 신년인사회에 참석해 발언하고 있다 사진 김포시 파이낸셜뉴스 김포 노진균 기자 경기 김포시가 기업지원 행정 자금지원 기술지원 마케팅지원 기업SOS 지원 기업지원 기반 조성 등 5개 분야로 구성된 로드맵을 공개했다 13일 김포시에 따르면 시는 이달 10일 시는 올해 자금 기술 판로 ',
 '716#SK에너지 네이버 한진과 중소상공인 부담 더는 택배 서비스 개시 지난해 12월 서울 종로구 SK서린빌딩에서 열린 업무협약식에서 오종훈 오른쪽 SK에너지 P M CIC 대표와 이윤숙 네이버 포레스트 CIC대표가 체결서를 들어보이고 있다 사진제공 SK에너지 SK에너지가 네이버 한진과 함께 중소상공인 SME 의 물류 부담을 덜어주는 더 The 착한택배 서비스를 선보인다고 13일 밝혔다 더 The ',
 '719#DL이앤씨 탄소저감 경영 우수기업 선정 지난 10일 서울 여의도 글래드호텔에서 열린 탄소정보공개프로젝트 CDP 기후변화 대응 우수기업 시상식에서 김대호 DL이앤씨 품질환경 담당

토픽별로 핵심 기사들의 인덱스를 뽑고 제목, 날짜, 내용의 데이터프레임을 만들겠습니다.

In [106]:
results_df = []

for i in range(len(results)):
    key_idx = [] 
    for j in range(len(results[i])):
        key_idx.append(results[i][j][:results[i][j].find('#')])
    
    results_df.append(df.iloc[list(map(int,key_idx))])

0번째 토픽의 핵심 기사는 다음과 같습니다.

In [108]:
results_df[0][['일자','제목','본문','Topics','Name']]

,일자,제목,본문,Topics,Name
55,2023-02-12,구자열 LS그룹 의장 명예 철학박사,구자열 LS그룹 이사회 의장 한국무역협회 회장 사진 이 모교인 고려대에서 경영인이 ...,0,0_학위_승진_ls그룹_서울
1733,2023-02-13,BNK금융 계열사 CEO 후보군 선정 안감찬 이두호 용퇴,사진 BNK금융그룹 BNK금융그룹은 주요 자회사인 부산은행 경남은행 BNK캐피탈 ...,0,0_학위_승진_ls그룹_서울
2057,2023-02-13,김포시 지역 경제 견인할 기업지원 로드맵 공개,김병수 김포시장이 지난달 13일 진행된 김포상공회의소 신년인사회에 참석해 발언하고 ...,0,0_학위_승진_ls그룹_서울
716,2023-02-13,SK에너지 네이버 한진과 중소상공인 부담 더는 택배 서비스 개시,지난해 12월 서울 종로구 SK서린빌딩에서 열린 업무협약식에서 오종훈 오른쪽 SK에...,0,0_학위_승진_ls그룹_서울
719,2023-02-13,DL이앤씨 탄소저감 경영 우수기업 선정,지난 10일 서울 여의도 글래드호텔에서 열린 탄소정보공개프로젝트 CDP 기후변화 대...,0,0_학위_승진_ls그룹_서울
1900,2023-02-13,CJ대한통운 신입사원 환영파티 열어 첫 일정 인디락 콘서트,헤럴드경제 김성우 기자 지난해 새롭게 종로시대 를 시작한 CJ대한통운이 사옥을 이...,0,0_학위_승진_ls그룹_서울


![img](./image/topic0.png)

1번째 토픽의 핵심 기사는 다음과 같습니다.

In [109]:
results_df[1][['일자','제목','본문','Topics','Name']]

,일자,제목,본문,Topics,Name
2330,2023-02-14,이자수익 40조 은행권 압박한 돈잔치 안돼,윤석열 대통령은 13일 은행 고금리로 인해 국민들 고통이 크다 며 은행의 돈잔치 로...,1,1_대통령_윤_은행_국민
2469,2023-02-14,윤 대통령 고금리 큰 고통 은행 돈 잔치 대책 세워라,앵커 윤석열 대통령이 은행의 고금리로 국민이 고통받고 있다며 당국에 대책 마련을 ...,1,1_대통령_윤_은행_국민
1502,2023-02-13,4대 금융지주 이자수익 2022년 40조 성과 퇴직금 펑펑 금융권 지배구조 개선 본격화,윤석열 대통령의 은행이 돈잔치를 벌이고 있다 는 지적은 최근 기준금리 상승으로 은행...,1,1_대통령_윤_은행_국민
1726,2023-02-13,성장 가속도 붙은 경제지표 무착륙 시나리오 급부상,지난달 고용 소비 기대이상 강세 연준 내부 경제성장 가속 붙어 연 경착륙 아닌 노랜...,1,1_대통령_윤_은행_국민
499,2023-02-12,주간증시전망 미국 CPI 코스피 향방 좌우 2500 저항선 뚫을까,아시아경제 이선애 기자 이번 주 13일 17일 국내 주식시장은 14일 발표되는 미...,1,1_대통령_윤_은행_국민
1706,2023-02-13,금감원 은행 결산 현장검사 착수 배당 성향 적절여부 따진다,머니투데이 이용안 기자 금융감독원이 은행을 대상으로 결산 현장검사에 돌입했다 매년...,1,1_대통령_윤_은행_국민


2번째 토픽의 핵심 기사는 다음과 같습니다.

In [110]:
results_df[2][['일자','제목','본문','Topics','Name']]

,일자,제목,본문,Topics,Name
1184,2023-02-13,현대차그룹 전기차 누적판매 10만대 돌파 최고 인기차는,헤럴드경제 김성우 기자 현대자동차그룹이 지난 2014년 미국 시장에서 전기차 EV...,2,2_전기차_기아_판매량_자동차
1447,2023-02-13,테슬라 2위로 밀어낸 전기차 현대차 기아는 글로벌 6위,작년 전 세계 전기차 판매량이 1000만대를 돌파한 가운데 현대자동차 기아가 글로벌...,2,2_전기차_기아_판매량_자동차
236,2023-02-12,JD파워 품질조사 1위 현대차그룹 1월 10만대 판매 돌파,현대자동차그룹이 미국 최고 권위의 품질조사에서 자동차그룹 기준 2년 연속 1위를 달...,2,2_전기차_기아_판매량_자동차
50,2023-02-12,전문기자 칼럼 포니로 시작한 현대차 신화 세계 1위도 꿈 아냐,현대차는 지난해 최대 자동차시장인 북미에서 5 6 의 점유율을 기록했다 기아까지 합...,2,2_전기차_기아_판매량_자동차
414,2023-02-12,독일 브랜드 안 보이네 국내 전기차 시장 현기 테슬라가 싹쓸이,국내 자동차 시장에서 강력한 존재감을 발휘하는 수입차 브랜드들이 전기차 시장에서는 ...,2,2_전기차_기아_판매량_자동차
1245,2023-02-13,진격의 전기차 1000만대 시대 활짝,중국 전기차 업체 BYD가 지난해 전 세계에서 전기차를 가장 많이 판매한 회사로 꼽...,2,2_전기차_기아_판매량_자동차


3번째 토픽의 핵심 기사는 다음과 같습니다.

In [111]:
results_df[3][['일자','제목','본문','Topics','Name']]

,일자,제목,본문,Topics,Name
100,2023-02-12,챗GPT가 불지핀 AI 고도화 경쟁 초당 1경 7100조번 연산 슈퍼컴 구축한 SKT,미국 인공지능 AI 연구소 오픈AI의 채팅봇 챗GPT 의 세계적 흥행을 계기로 글로...,3,3_ai_인공지능_반도체_sk텔레콤
1984,2023-02-13,SKT 초거대AI용 슈퍼컴 용량 확대,SK텔레콤은 인공지능 AI 서비스 에이닷 의 두뇌 역할을 하는 슈퍼컴퓨터 타이탄 을...,3,3_ai_인공지능_반도체_sk텔레콤
246,2023-02-12,기획 챗GPT발 AI인프라 투자 불붙었다,대화형 AI 인공지능 챗GPT 가 촉발한 기술 전쟁에 국내 대표 IT 기업들이 참전...,3,3_ai_인공지능_반도체_sk텔레콤
2,2023-02-12,초거대 AI 두뇌 2배로 확대 SKT 한국형 챗GPT 속도전,인공지능 AI 연구 스타트업 오픈AI의 챗GPT 공개 이후 국내외 빅테크의 차세대 ...,3,3_ai_인공지능_반도체_sk텔레콤
2171,2023-02-13,너도나도 챗GPT 달려간다 상한가 치솟은 이 종목,오픈엣지테크놀로지 상한가 인공지능 반도체 기대 커지며 반도체 IP 디자인하우스 각광...,3,3_ai_인공지능_반도체_sk텔레콤
1272,2023-02-13,SKT AI 인프라 확대 에이닷 고도화,SK텔레콤의 초거대 인공지능 AI 서비스인 에이닷 을 제어하는 슈퍼컴퓨터 타이탄 의...,3,3_ai_인공지능_반도체_sk텔레콤


#### 토픽별 핵심 기사 다시 합쳐서 저장

In [112]:
concat_results_df = pd.concat(results_df)

In [113]:
concat_results_df

,일자,제목,본문,Topics,Topic,Count,Name
55,2023-02-12,구자열 LS그룹 의장 명예 철학박사,구자열 LS그룹 이사회 의장 한국무역협회 회장 사진 이 모교인 고려대에서 경영인이 ...,0,0,588,0_학위_승진_ls그룹_서울
1733,2023-02-13,BNK금융 계열사 CEO 후보군 선정 안감찬 이두호 용퇴,사진 BNK금융그룹 BNK금융그룹은 주요 자회사인 부산은행 경남은행 BNK캐피탈 ...,0,0,588,0_학위_승진_ls그룹_서울
2057,2023-02-13,김포시 지역 경제 견인할 기업지원 로드맵 공개,김병수 김포시장이 지난달 13일 진행된 김포상공회의소 신년인사회에 참석해 발언하고 ...,0,0,588,0_학위_승진_ls그룹_서울
716,2023-02-13,SK에너지 네이버 한진과 중소상공인 부담 더는 택배 서비스 개시,지난해 12월 서울 종로구 SK서린빌딩에서 열린 업무협약식에서 오종훈 오른쪽 SK에...,0,0,588,0_학위_승진_ls그룹_서울
719,2023-02-13,DL이앤씨 탄소저감 경영 우수기업 선정,지난 10일 서울 여의도 글래드호텔에서 열린 탄소정보공개프로젝트 CDP 기후변화 대...,0,0,588,0_학위_승진_ls그룹_서울
...,...,...,...,...,...,...,...
2451,2023-02-14,제로슈거 소주 처음처럼 새로 다섯달만에 누적판매 5천만병,롯데칠성음료가 출시한 처음처럼 새로 가 소주업계에 돌풍을 일으키고 있다 13일 롯데...,65,65,11,65_밀키스_제로_칼로리_롯데칠성음료
2431,2023-02-14,GS25 딸바 막걸리 출시 MZ세대 취향저격,GS25는 국내 대표 전통주 술도가 남한산성소주와 손잡고 딸바 막걸리 를 업계 단독...,65,65,11,65_밀키스_제로_칼로리_롯데칠성음료
1710,2023-02-13,기회의 창업사관학교 윤여진 카페매직샵 대표 케이크 시트부터 토핑까지 원하는대로 소...,디저트 dessert 는 양식에서 식사 끝에 나오는 과자나 과일 등으로 이뤄진 코스...,65,65,11,65_밀키스_제로_칼로리_롯데칠성음료
2136,2023-02-13,사랑해요 밀키스 34년 만에 새 제품 제로 칼로리 내놓았다,30년 넘게 사랑받은 밀키스가 제로 칼로리 음료로 나왔다 롯데칠성음료가 1989년 ...,65,65,11,65_밀키스_제로_칼로리_롯데칠성음료


In [114]:
file_path =  "./data/"

file_name = "한국뉴스"+'_토픽별_핵심기사추출' + ".csv"

file_full_path = file_path + file_name

concat_results_df.to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜto_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜto_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜ 

# 2. 토픽별 클러스터링 된 기사 각각 요약

## 2-2. 핵심 문장 추출 
KR-WordRank 에서는 keywords 를 많이 포함한 문장을 핵심 문장으로 선택한다. 문장을 추출하는 원리는 추출된 키워드의 랭크값을 이용하여 키워드 벡터를 만든 뒤, 코싸인 유사도 기준으로 입력된 문장 벡터가 키워드 벡터와 유사한 문장을 선택하는 것이다.
summarize_with_sentences 함수에 기사 내용 전체 를 입력하면 KR-WordRank 를 학습하여 키워드와 이를 이용한 핵심 문장을 선택한다. 

몇 가지 parameter를 추가할 수 있다. stopwords 는 키워드에서 제거한다. 이들은 키워드 벡터를 만들 때에도 이용되지 않는다. 
또한 키워드 벡터와 유사한 문장을 우선적으로 선택하다 보면 이전에 선택된 문장과 중복되는 문장들이 선택되기도 하는데 이는 diversity 를 이용하여 조절할 수 있다. diversity 는 코사인 유사도 기준 핵심 문장 간의 최소 거리이며 이 값이 클수록 다양한 문장이 선택된다. 

In [115]:
from krwordrank.sentence import summarize_with_sentences

stopwords =' 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓'

def summarize_article(texts):

    keywords, sents = summarize_with_sentences(texts, 
                                               stopwords=stopwords, 
                                               num_keywords=10, 
                                               num_keysents=3,  
                                               diversity=0.5)
    
    return sents

In [116]:
concat_results_df['ArticleSummary'] = ''

기사를 문장 단위로 자르기 위해 '다.'으로 나눈뒤 다시 '다.'를 문장 끝에 붙여줍니다

In [59]:
# [x+'다.' for x in concat_results_df.Article.iloc[i].split('다.')]

In [145]:
for i in range(len(concat_results_df)):
    article = [x for x in concat_results_df.본문.iloc[i].split('다') if x is not '']
    
    try:   
        concat_results_df['ArticleSummary'].iloc[i] = summarize_article(article)
        
    except:
         concat_results_df['ArticleSummary'].iloc[i] = article

<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30212\2607906888.py:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  article = [x for x in concat_results_df.본문.iloc[i].split('다') if x is not '']


문장별로 나뉜 결과를 다시 하나로 합쳐줍니다.

In [146]:
def concat_sent(sent_list):
    return " ".join(sent_list)
    
concat_results_df['ArticleSummary'] = concat_results_df['ArticleSummary'].apply(concat_sent)


토픽0의 핵심기사들의 제목과 요약문입니다

In [147]:
for index, row in concat_results_df[concat_results_df.Topics==0].iterrows():
    print('기사 제목:',row.제목,'\n','기사 내용:',row.ArticleSummary,'\n')

기사 제목: 구자열 LS그룹 의장 명예 철학박사 
 기사 내용: 구자열 LS그룹 이사회 의장 한국무역협회 회장 사진 이 모교인 고려대에서 경영인이 받을 수 있는 최고 영예인 명예철학박사 학위를 받았  12일 LS에 따르면 고려대는 지난 10일 서울 성북구 고려대 100주년기념삼성관에서 정진택 총장 이관영 대학원장을 비롯한 관계자들이 참석한 가운데 구 의장에게 명예철학박사 학위를 수여했  서진우 기자  

기사 제목:  BNK금융 계열사 CEO 후보군 선정 안감찬 이두호 용퇴  
 기사 내용:  사진 BNK금융그룹 BNK금융그룹은 주요 자회사인 부산은행 경남은행 BNK캐피탈 최고경영자 CEO 선임을 위한 임원후보추천위원회 임추위 를 개최해 1차 후보군을 선정했 고 13일 밝혔  BNK에 따르면 이들 자회사는 앞서 지난 10일까지 최고경영자 후보군을 대상으로 지원서 제출을 요청한 바 있  임추위는 지원서를 제출한 후보  

기사 제목: 김포시 지역 경제 견인할 기업지원 로드맵 공개 
 기사 내용: 김병수 김포시장이 지난달 13일 진행된 김포상공회의소 신년인사회에 참석해 발언하고 있  사진 김포시 파이낸셜뉴스 김포 노진균 기자 경기 김포시가 기업지원 행정 자금지원 기술지원 마케팅지원 기업SOS 지원 기업지원 기반 조성 등 5개 분야로 구성된 로드맵을 공개했  13일 김포시에 따르면 시는 이달 10일 시는 올해 자금 기술 판로  

기사 제목: SK에너지 네이버 한진과 중소상공인 부담 더는 택배 서비스 개시 
 기사 내용: 지난해 12월 서울 종로구 SK서린빌딩에서 열린 업무협약식에서 오종훈 오른쪽 SK에너지 P M CIC 대표와 이윤숙 네이버 포레스트 CIC대표가 체결서를 들어보이고 있  사진제공 SK에너지 SK에너지가 네이버 한진과 함께 중소상공인 SME 의 물류 부담을 덜어주는 더 The 착한택배 서비스를 선보인 고 13일 밝혔  더 The  

기사 제목: DL이앤씨 탄소저감 경영 우수기업 선정 
 기사 내용: 지난 10일 서울 여의도 글래드호텔에서 열린 탄소

In [149]:
file_path =  "./data/"

file_name = "한국뉴스_" + '토픽별_핵심기사추출_요약' + ".csv"

file_full_path = file_path + file_name

concat_results_df.to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜto_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜto_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜ 

<hr><font color="green"><h1>from file: 3.팔로업 뉴스 추천 시스템</h1></font>

## 1. 코사인 유사도

[참조]
코사인 유사도 개념 : https://thecleverprogrammer.com/2021/02/27/cosine-similarity-in-machine-learning/


코사인 유사도는 두 개의 벡터값에서 코사인 각도를 구하는 방법이다. -1~1 사이의 값을 가지고 1에 가까울수록 유사하다는 것을 의미한다. 이 유사도가 많이 사용되는 이유는 단손히 좌표상의 거리를 구하는 것이 아니라 두 벡터간의 각도를 구하는 것이기 때문에 방향성이 더해지기 때문이다.


코사인 유사성은 두 문서 간의 유사성을 찾는 데 사용된다. 벡터 간의 유사도 점수를 계산하여 이를 수행하며, 이는 벡터 사이의 각도를 찾아 수행된다. 유사도의 범위는 0에서 1 사이다. 두 벡터 간의 유사도 점수 값이 1이면 두 벡터 간의 유사도가 더 크다는 의미이다. 반면에 두 벡터 간의 유사도 점수 값이 0이면 두 벡터 사이에 유사도가 없음을 의미한다. 유사도 점수가 1이면 두 벡터 사이의 각도는 0이고 유사도 점수가 0이면 두 벡터 사이의 각도는 90도이다. 이 기술은 더 나은 사용자 경험을 제공하기 위해 가장 유사한 제품을 사용자에게 추천할 수 있도록 두 제품 설명 간의 유사점을 찾기 위해 추천 시스템에서 주로 사용된다.

## 2. 기사 추천 시스템 

[참조]

기계 학습을 통한 기사 추천 시스템 : https://thecleverprogrammer.com/2021/11/10/article-recommendation-system-with-machine-learning/

문서의 행렬 표현 BoW, DTM, TF-IDF : https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=shino1025&logNo=221868521724

### 2-1.	추천 시스템 원리 

추천 시스템을 만드는 방법에는 여러 가지가 있다. 기사 추천 시스템을 만들기 위해서는 사용자의 관심보다 콘텐츠에 집중해야 한다. 예를 들어 사용자가 클러스터링을 기반으로 한 기사를 읽으면 모든 권장 기사도 클러스터링을 기반으로 해야 한다. 따라서 콘텐츠를 기반으로 기사를 추천하려면
1.	기사의 내용을 이해해야한다
2.	내용을 다른 모든 기사와 일치시켜야 한다.
3.	독자가 이미 읽고 있는 기사에 가장 적합한 기사를 추천한다.

scikit-Learn에서 제공하는 "cosine_similarity" 방법을 사용할 수 있다. 두 개의 배열을 만든 다음 Scikit-Learn 라이브러리에서 제공하는 "cosine_similarity" 메서드를 구현하여 둘 사이의 유사점을 찾는 것이다. 다음은 코사인 유사도를 계산하는 방법이다.


In [150]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

In [152]:
df=pd.read_excel('빅카인즈_뉴스.xlsx')
df=df.loc[:,['일자','제목','본문']]

In [154]:
df['Recommended Index'] = ''

### 2-2.	TF-IDF¶

tf-idf(Term Frequency - Inverse Document Frequency)? 

TF(단어빈도)는 특정한 단어가 문서 내에서 얼마나 자주 등장하는지를 나타내는 값이다. 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 하나의 문서에서 많이 나오지 않고 다른 문서에서 자주 등장하면 단어의 중요도는 낮아진다. 이를 DF(문서빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값으로 점수가 높은 단어일수록 다른 문서에는 많이 않고 해당 문서에서 자주 등장하는 단어를 의미한다.


In [155]:
articles = df["본문"].tolist()

uni_tfidf = text.TfidfVectorizer(input=articles) 
uni_matrix = uni_tfidf.fit_transform(articles) # 학습
uni_sim = cosine_similarity(uni_matrix)

In [156]:
uni_matrix.toarray().shape


(17331, 118674)

In [157]:
uni_sim.shape


(17331, 17331)

특정 기사와 같은 날짜의 기사는 추천하지 않도록 설정했다 왜냐하면 매우 유사한 내용의 뉴스일 확률이 높기 때문이다.

그리고 후속기사(팔로업뉴스)를 추천하기 위해서는 특정 기사 다음 날 부터 관련 기사 추출해야 함

In [159]:
df.iloc[uni_sim[0].argsort()[::-1][1:5], : ]['일자']

78       20230214
16861    20230207
13762    20230208
3220     20230213
Name: 일자, dtype: int64

In [ ]:
# # 같은 날짜의 기사는 추천하지 않도록 코드 만들기

# a0 = df.iloc[uni_sim[1470].argsort()[::-1][0], : ]['Datetime']

# a = df.iloc[uni_sim[1470].argsort()[::-1][1:5], : ]['Datetime']

# b = a.index[df.iloc[uni_sim[1470].argsort()[::-1][0], :]['Datetime'] < a]

# print(a0,'\n')
# print(a,'\n')
# print(b,'\n')

In [160]:
def recommend_articles_index(xx):
        a = df.iloc[xx.argsort()[::-1][1:6], : ]['일자'] # 추천 기사들의 날짜들 

        b = a.index[(df.iloc[xx.argsort()[::-1][0], :]['일자'] < a) ] # 같은 날짜는 추천에서 제외

        return b.tolist()

df['Recommended Index'] = [recommend_articles_index(x) for x in uni_sim]

특정 기사의 날짜 이후로 가장 유사도 높은 기사들의 제목과 링크와 날짜를 함께 보여줌

가장 최신 날짜(2022.07.17)의 기사는 팔로업 기사가 존재하지 않음

In [162]:
df.일자=df.일자.astype(str)

In [163]:
print('특정 기사: \n',  df['제목'].iloc[1]+' '+ df['일자'].iloc[0] + '\n')

print('추천 기사: \n', df['Recommended Index'][1])

특정 기사: 
 필옵틱스, 지난해 영업익 177억 흑자전환 20230214

추천 기사: 
 []


1427번 기사를 예시로 보겠습니다.

In [164]:
k = 755
print('특정 기사: \n',  df['제목'].iloc[k]+' '+ df['일자'].iloc[k] + '\n')
print('팔로업 기사: \n', df.iloc[df['Recommended Index'][k]][['제목', '일자']].values)

특정 기사: 
 '보톡스 전쟁' 관련 첫 판결에 국내 업체들 초긴장...7조 원 시장에 지각변동 또 오나 20230213

팔로업 기사: 
 [['보톡스 소송 6년 만에 첫 판결 시장판도 바뀌나' '20230214']]


1260번 기사의 팔로업 기사들을 출력합니다.

In [165]:
k = 1260
print('특정 기사: \n',  df['제목'].iloc[k]+' '+ df['일자'].iloc[k] + '\n')
print('팔로업 기사: \n', df.iloc[df['Recommended Index'][k]][['제목', '일자']].values)

특정 기사: 
 원희룡 “초소형 행복주택, 세대통합 리모델링으로 공실 해소하겠다” 20230213

팔로업 기사: 
 [['원희룡 “좁은 면적 하나로 합쳐 평수 확대 초소형 행복주택 공실 해소”' '20230214']]


In [167]:
df.iloc[1260]['본문']

'정부가 생활이 불편할 정도로 좁은 행복주택을 개조해 수요자 눈높이에 맞춘다. 이미 조성된 행복주택은 두 세대를 하나로 합쳐 면적을 넓힌다. 상대적으로 저렴한 주거비와 안정적인 거주기간을 보장하는데도 ‘초소형’이란 조건 때문에 외면 받고 있다는 판단에서다. \n \n\n 원희룡 국토교통부 장관은 13일 오후 2시 화성동탄에 위치한 한국토지주택공사(LH) 행복주..'

![img](./image/1260_fol.png)


In [168]:
file_path =  "./data/"

file_name = '한국뉴스_팔로업_뉴스_추천시스템' + ".csv"

file_full_path = file_path + file_name

df.to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜto_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜto_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜ 



# 팔로업 뉴스의 요약

먼저 각각의 기사들을 각각 요약한다

In [169]:
from krwordrank.sentence import summarize_with_sentences

stopwords =' 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓'

def summarize_article(texts):

    keywords, sents = summarize_with_sentences(texts, 
                                               stopwords=stopwords, 
                                               num_keywords=20, 
                                               num_keysents=3,  
                                               diversity=0.5)
    
    return sents

In [170]:
df['ArticleSummary'] = ''

In [183]:
for i in range(len(df)):
    article = [x+'다' for x in df.본문.iloc[i].split('다') if x != '']
    try:
        df['ArticleSummary'].iloc[i] = summarize_article(article)        
    except: 
        df['ArticleSummary'].iloc[i] = article
        

문장별로 나뉜 결과를 다시 하나로 합쳐줍니다.

In [184]:
def concat_sent(sent_list):
    return " ".join(sent_list)
    
df['ArticleSummary'] = df['ArticleSummary'].apply(concat_sent)



1260번 기사의 요약 결과는 다음과 같습니다.

In [194]:
#마지막 문장의 '다' 제거
df.ArticleSummary=df.ArticleSummary.apply(lambda x : x.rstrip('다'))

In [185]:
df['ArticleSummary'].iloc[1260]

'정부가 생활이 불편할 정도로 좁은 행복주택을 개조해 수요자 눈높이에 맞춘다 . 이미 조성된 행복주택은 두 세대를 하나로 합쳐 면적을 넓힌다 . 상대적으로 저렴한 주거비와 안정적인 거주기간을 보장하는데도 ‘초소형’이란 조건 때문에 외면 받고 있다 는 판단에서다 . \n \n\n 원희룡 국토교통부 장관은 13일 오후 2시 화성동탄에 위치한 한국토지주택공사(LH) 행복주..다'

![img](./image/1260_summary.png)


1260번 기사의 팔로업 뉴스들을 각각 요약한 결과입니다.

In [186]:
for index, row in df.iloc[df['Recommended Index'][1260]].iterrows():
    print('기사 제목:',row.제목,'\n','기사 내용:',row.ArticleSummary,'\n')

기사 제목: 원희룡 “좁은 면적 하나로 합쳐 평수 확대 초소형 행복주택 공실 해소” 
 기사 내용: 정부가 규모가 협소한 공공임대주택 두 가구를 하나로 합치는 리모델링을 통해 평수를 넓히는 방안을 추진한다 . 늘어가는 행복주택 공실을 해소하기 위한 조치다 . 
 원희룡 국토교통부 장관이 13일 오후 경기도 화성시 동탄에 위치한 한국토지주택공사(LH) 행복주택을 방문, 초소형 평형(20㎡ 이하)의 공실 현황을 점검하고 있다 . 국토교통부 제공 원희룡 국토교통..다 



![img](./image/1260_all_sum.png)


In [196]:
file_path =  "./data/"

file_name = "한귝뉴스_" + '팔로업_뉴스_추천시스템_요약' + ".csv"

file_full_path = file_path + file_name

df.to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜto_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜ 


